# Data Quality Monitoring Test

## Overview
This notebook tests the **Data Quality (DQ) automation** setup using Snowflake's **Data Metric Functions (DMFs)** and alerts.

### What We're Testing:
1. ✅ **DMF Configuration** - Verify DMFs are monitoring NULL values
2. ✅ **Baseline Data** - Insert valid records and confirm 0 NULLs detected
3. ⚠️ **Problematic Data** - Insert records with NULL values
4. 🚨 **Alert Triggering** - Verify alerts fire when NULLs detected
5. 📊 **Results Visualization** - View DQ metrics and alert history

### Architecture:
```
Raw Tables (orders, order_items)
    ↓ DML operations (INSERT/UPDATE/DELETE)
DMFs Execute (TRIGGER_ON_CHANGES)
    ↓ Store results
SNOWFLAKE.LOCAL.DATA_QUALITY_MONITORING_RESULTS
    ↓ Alert queries every 5 minutes
Alert Fires → data_quality_alerts table
```

In [ ]:
-- Setup: Set context
USE DATABASE automated_intelligence;
USE SCHEMA raw;
USE WAREHOUSE automated_intelligence_wh;

---
## Step 1: Current State Analysis

Let's check the current state of our data and alerts before we begin testing.

In [ ]:
-- Check current record counts
SELECT 'Orders' AS table_name, COUNT(*) AS record_count FROM orders
UNION ALL
SELECT 'Order Items', COUNT(*) FROM order_items
UNION ALL
SELECT 'DQ Alerts', COUNT(*) FROM data_quality_alerts;

---
## Step 2: Verify DMF Configuration

Check which columns are being monitored and which columns allow NULL values.

In [ ]:
-- Identify columns that allow NULLs (these are what we'll test)
SELECT 
  table_name,
  column_name,
  CASE WHEN is_nullable = 'YES' THEN '✅ Allows NULL' ELSE '❌ NOT NULL' END AS nullable_status,
  data_type
FROM INFORMATION_SCHEMA.COLUMNS
WHERE table_schema = 'RAW'
  AND table_name IN ('ORDERS', 'ORDER_ITEMS')
ORDER BY table_name, ordinal_position;

In [ ]:
-- Verify alert is active
SHOW ALERTS LIKE 'data_quality_alert' IN SCHEMA automated_intelligence.raw;

---
## Step 3: Insert Valid Baseline Data

First, let's insert **clean, valid data** to establish a baseline. DMFs should detect **0 NULL values** for these records.

In [ ]:
-- Insert valid orders (no NULLs)
INSERT INTO orders (order_id, customer_id, order_date, order_status, total_amount, discount_percent, shipping_cost)
VALUES 
  (999990, 1, CURRENT_TIMESTAMP(), 'Pending', 100.00, 0, 5.00),
  (999991, 2, CURRENT_TIMESTAMP(), 'Pending', 200.00, 10, 10.00);

SELECT 'Valid orders inserted successfully' AS status;

In [ ]:
-- Insert valid order items (no NULLs)
INSERT INTO order_items (order_item_id, order_id, product_id, product_name, product_category, quantity, unit_price, line_total)
VALUES 
  (999990, 999990, 5000, 'Test Product A', 'Electronics', 1, 95.00, 95.00),
  (999991, 999991, 5001, 'Test Product B', 'Clothing', 2, 95.00, 190.00);

SELECT 'Valid order items inserted successfully' AS status;

In [ ]:
-- Wait for DMFs to execute (they run on TRIGGER_ON_CHANGES)
CALL SYSTEM$WAIT(5);
SELECT '⏳ Waiting 5 seconds for DMFs to process baseline data...' AS status;

---
## Step 4: Check DMF Results (Baseline)

Query the DMF results to confirm **0 NULL values** detected for our valid data.

In [ ]:
-- Check recent DMF results
SELECT 
  measurement_time,
  table_name,
  column_name,
  metric_name,
  value AS null_count,
  CASE WHEN value = 0 THEN '✅ Clean' ELSE '⚠️ NULLs Found' END AS status
FROM SNOWFLAKE.LOCAL.DATA_QUALITY_MONITORING_RESULTS
WHERE 
  table_database = 'AUTOMATED_INTELLIGENCE'
  AND table_schema = 'RAW'
  AND table_name IN ('ORDERS', 'ORDER_ITEMS')
  AND measurement_time >= DATEADD('MINUTE', -5, CURRENT_TIMESTAMP())
ORDER BY measurement_time DESC, table_name, column_name;

---
## Step 5: Insert Problematic Data 🚨

Now let's insert **records with NULL values** in columns that allow NULLs. This will trigger the DMFs and (eventually) the alert.

In [ ]:
-- Insert orders with NULLs in nullable columns
INSERT INTO orders (order_id, customer_id, order_date, order_status, total_amount, discount_percent, shipping_cost)
VALUES 
  (999992, NULL, CURRENT_TIMESTAMP(), 'Pending', 175.00, NULL, NULL),  -- NULL customer_id, discount, shipping
  (999993, NULL, NULL, 'Pending', 225.00, NULL, NULL);  -- Multiple NULLs

SELECT '⚠️ Problematic orders inserted (with NULLs)' AS status;

In [ ]:
-- Insert order items with NULLs in nullable columns
INSERT INTO order_items (order_item_id, order_id, product_id, product_name, product_category, quantity, unit_price, line_total)
VALUES 
  (999992, 999992, NULL, 'Test Product C', NULL, NULL, 50.00, NULL),  -- Multiple NULLs
  (999993, 999993, NULL, NULL, NULL, NULL, NULL, NULL);  -- Extreme case - many NULLs

SELECT '⚠️ Problematic order items inserted (with NULLs)' AS status;

In [ ]:
-- Wait for DMFs to process problematic data
CALL SYSTEM$WAIT(10);
SELECT '⏳ Waiting 10 seconds for DMFs to detect NULLs...' AS status;

---
## Step 6: DMF Results - NULL Detection

Check DMF results again. We should now see **NULL counts > 0** for the problematic records.

In [ ]:
-- View DMF results showing NULL values detected
SELECT 
  measurement_time,
  table_name,
  column_name,
  metric_name,
  value AS null_count,
  '⚠️ NULLs Detected!' AS status
FROM SNOWFLAKE.LOCAL.DATA_QUALITY_MONITORING_RESULTS
WHERE 
  table_database = 'AUTOMATED_INTELLIGENCE'
  AND table_schema = 'RAW'
  AND table_name IN ('ORDERS', 'ORDER_ITEMS')
  AND value > 0  -- Only show columns with NULL values
ORDER BY measurement_time DESC, table_name, column_name
LIMIT 20;

In [ ]:
# Visualize NULL counts by column
import altair as alt

# Get the data from previous query
df = dmf_results_with_nulls.to_pandas()

if not df.empty:
    # Create bar chart
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X('NULL_COUNT:Q', title='Number of NULL Values'),
        y=alt.Y('COLUMN_NAME:N', title='Column', sort='-x'),
        color=alt.Color('TABLE_NAME:N', title='Table'),
        tooltip=['TABLE_NAME', 'COLUMN_NAME', 'NULL_COUNT']
    ).properties(
        title='NULL Values Detected by Column',
        width=600,
        height=300
    )
    
    chart
else:
    st.warning("No NULL values detected yet. DMFs may still be processing.")

---
## Step 7: Alert Condition Preview

Let's manually run the **alert condition query** to see what the alert will detect. The actual alert runs every 5 minutes automatically.

In [ ]:
-- Manually execute the alert condition query
SELECT 
  measurement_time,
  table_name,
  metric_name,
  column_name,
  value AS null_count,
  '🚨 This will trigger an alert' AS alert_trigger
FROM SNOWFLAKE.LOCAL.DATA_QUALITY_MONITORING_RESULTS
WHERE 
  table_database = 'AUTOMATED_INTELLIGENCE'
  AND table_schema = 'RAW'
  AND table_name IN ('ORDERS', 'ORDER_ITEMS')
  AND metric_name = 'NULL_COUNT'
  AND value > 0
  AND measurement_time >= DATEADD('MINUTE', -10, CURRENT_TIMESTAMP())
ORDER BY measurement_time DESC;

---
## Step 8: Wait for Alert Execution

The alert `data_quality_alert` runs **every 5 minutes**. Let's wait and then check if it fired.

⏰ **Note**: You may need to wait up to 5 minutes for the alert to execute and log results to the `data_quality_alerts` table.

In [ ]:
-- Wait 30 seconds (alert may have already run or will run soon)
CALL SYSTEM$WAIT(30);
SELECT '⏳ Waiting for alert to execute... (runs every 5 minutes)' AS status;

---
## Step 9: Check Alert History

Query the `data_quality_alerts` table to see if alerts have been logged.

In [ ]:
-- Check if alerts were fired
SELECT 
  alert_time,
  issue_summary,
  '🚨 Alert fired!' AS status
FROM data_quality_alerts
ORDER BY alert_time DESC
LIMIT 10;

In [ ]:
# Visualize alert timeline
import altair as alt
import pandas as pd

df_alerts = check_alerts.to_pandas()

if not df_alerts.empty:
    # Create timeline chart
    chart = alt.Chart(df_alerts).mark_circle(size=100).encode(
        x=alt.X('ALERT_TIME:T', title='Alert Time'),
        y=alt.Y('ISSUE_SUMMARY:N', title='Issue'),
        color=alt.value('red'),
        tooltip=['ALERT_TIME', 'ISSUE_SUMMARY']
    ).properties(
        title='Data Quality Alerts Timeline',
        width=700,
        height=200
    )
    
    chart
    
    st.success(f"✅ {len(df_alerts)} alert(s) fired!")
else:
    st.info("⏰ No alerts fired yet. The alert runs every 5 minutes. Check back soon!")

---
## Step 10: Alert Execution History

View detailed alert execution history to see when the alert ran and whether the condition was true.

In [ ]:
-- Check alert execution history
SELECT 
  name AS alert_name,
  state,
  scheduled_time,
  completed_time,
  CASE 
    WHEN condition_evaluates_to_true = TRUE THEN '🚨 Condition TRUE (Alert Fired)'
    ELSE '✅ Condition FALSE (No Issues)'
  END AS condition_status,
  error
FROM TABLE(INFORMATION_SCHEMA.ALERT_HISTORY(
  SCHEDULED_TIME_RANGE_START => DATEADD('hour', -1, CURRENT_TIMESTAMP())
))
WHERE alert_name = 'DATA_QUALITY_ALERT'
ORDER BY scheduled_time DESC
LIMIT 10;

---
## Step 11: Cleanup (Optional)

Remove test data to reset the environment. **Uncomment and run if you want to clean up.**

In [ ]:
-- Cleanup test data (UNCOMMENT TO EXECUTE)
/*
DELETE FROM order_items WHERE order_item_id >= 999990;
DELETE FROM orders WHERE order_id >= 999990;
DELETE FROM data_quality_alerts WHERE alert_time >= DATEADD('hour', -1, CURRENT_TIMESTAMP());
SELECT '✅ Test data cleaned up' AS status;
*/

---
## 📊 Summary

### What We Tested:
1. ✅ **Valid Data** - Inserted clean records, confirmed 0 NULLs detected
2. ⚠️ **Problematic Data** - Inserted records with NULL values
3. 🔍 **DMF Detection** - DMFs automatically detected NULL values
4. 🚨 **Alert Firing** - Alert checked every 5 minutes and logged issues

### Key Monitoring Queries:

**Check DMF Results:**
```sql
SELECT * 
FROM SNOWFLAKE.LOCAL.DATA_QUALITY_MONITORING_RESULTS 
WHERE value > 0
ORDER BY measurement_time DESC;
```

**Check Alert History:**
```sql
SELECT * 
FROM data_quality_alerts 
ORDER BY alert_time DESC;
```

**Check Alert Status:**
```sql
SHOW ALERTS IN SCHEMA automated_intelligence.raw;
```

### Next Steps:
- Monitor `data_quality_alerts` table regularly
- Set up email/Slack notifications (requires notification integration)
- Add custom DMFs for business-specific validations
- Adjust alert frequency based on data ingestion patterns